In [5]:
m21_y = 14788800
m79_y = 16445000
m_diff = m79_y - m21_y

s21_y = 5569150
s79_y = 7556040
s_diff = s79_y - s21_y

print(f"March ice area difference: {m_diff} km2")
print(f"September ice area difference: {s_diff} km2")

March ice area difference: 1656200 km2
September ice area difference: 1986890 km2


In [4]:
a1 = -80976.9
b1 = 1.6843*10**8
ice_22_s = a1*2022 + b1
ice_23_s = a1*2023 + b1
ice_24_s = a1*2024 + b1
print(ice_22_s, ice_23_s, ice_24_s)

4694708.200000018 4613731.300000012 4532754.400000006


In [3]:
a2 = -39091.8
b2 = 9.3554*10**7
ice_22_m = a2*2022 + b2
ice_23_m = a2*2023 + b2
ice_24_m = a2*2024 + b2
print(ice_22_m, ice_23_m, ice_24_m)

14510380.399999991 14471288.599999994 14432196.799999997


In [8]:
year = (1000000-1.6843*10**8)/-80976.9
print(f"Year when the ice area will be 1 million km2: {year}")
years_left = year - 2024
print(f"Years left until the ice area will be 1 million km2: {years_left}")

Year when the ice area will be 1 million km2: 2067.626693538528
Years left until the ice area will be 1 million km2: 43.62669353852789
